In [1]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
from pyspark.sql import *
from pyspark.ml.feature import StopWordsRemover,RegexTokenizer, Word2Vec, Word2VecModel 
import pyspark.sql.functions as f
from pyspark.sql.types import *


In [2]:
# import os 
# memory = '20g'
# pyspark_submit_args = ' --driver-memory '+ memory + ' pyspark-shell'
# os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [3]:
spark = SparkSession \
        .builder \
        .appName("PA3") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

In [4]:
import pandas as pd
df = pd.read_csv('train.csv')
traindataframe = spark.createDataFrame(df)

df = pd.read_csv('test.csv')
testdataframe = spark.createDataFrame(df)

In [5]:
regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
traindataframe = regexTokenizer.transform(traindataframe)
testdataframe = regexTokenizer.transform(testdataframe)

swremover = StopWordsRemover(inputCol="words", outputCol="filteredwords")
traindataframe = swremover.transform(traindataframe)
testdataframe = swremover.transform(testdataframe)



In [6]:
# word2Vec = Word2Vec(inputCol="filteredwords", outputCol="features")

# trainmodel = word2Vec.fit(traindataframe)
# trainmodel.save('./Models/Word2Vec/Part3/train')
# traindataframe = trainmodel.transform(traindataframe)

# testmodel = word2Vec.fit(testdataframe)
# testmodel.save('./Models/Word2Vec/Part3/test')
# testdataframe = testmodel.transform(testdataframe)

In [7]:
trainmodel = Word2VecModel.load('./Models/Word2Vec/Part3/train')
traindataframe = trainmodel.transform(traindataframe)
testmodel = Word2VecModel.load('./Models/Word2Vec/Part3/test')
testdataframe = testmodel.transform(testdataframe)

In [8]:
traindataframe.select('movie_id','features','genre').show()

+--------+--------------------+--------------------+
|movie_id|            features|               genre|
+--------+--------------------+--------------------+
|23890098|[0.01131801471552...|['World cinema', ...|
|31186339|[0.02394582728102...|['Action/Adventur...|
|20663735|[0.01447957225656...|['Musical', 'Acti...|
| 2231378|[0.04825355684356...|          ['Comedy']|
|  595909|[0.00499553057240...|['Crime Fiction',...|
| 5272176|[-0.0092212500950...|['Action/Adventur...|
| 1952976|[-0.0131933859412...|['Thriller', 'Dra...|
|24225279|[0.01041678206258...|           ['Drama']|
| 2462689|[0.00843683906065...|['Black-and-white...|
|20532852|[-0.0038579294297...|['Animation', 'Sh...|
|15401493|[-0.0396562894587...|          ['Comedy']|
|18188932|[0.00636731243946...|['Crime Fiction',...|
| 2940516|[-0.0838465928921...|          ['Comedy']|
| 1480747|[0.03505517356264...|          ['Comedy']|
|24448645|[-0.0289078256223...|          ['Horror']|
|15072401|[-0.0369245740061...|['Crime Fiction

In [9]:
testdataframe.select('movie_id','features').show()

+--------+--------------------+
|movie_id|            features|
+--------+--------------------+
| 1335380|[-0.0127977656061...|
|29062594|[0.02603229098559...|
| 9252321|[0.00455041255011...|
|13455076|[0.00688066838351...|
|24165951|[-0.0543006666412...|
| 1925869|[-0.0156268229926...|
|10799612|[0.00824491203197...|
|28238240|[-0.0073060956783...|
|17124781|[-0.0337746019627...|
|28207941|[-0.0371017778940...|
|19174305|[-0.0090511224433...|
|18392317|[0.00799751521061...|
|34420857|[-0.0137272985654...|
| 4039635|[-0.0035953696568...|
| 8034072|[0.00386177607554...|
| 4016437|[-0.0046817680668...|
| 1520023|[-0.0177962624929...|
|24589422|[-0.0070355151931...|
|35068740|[5.04269166412996...|
|21132951|[-0.0147446817731...|
+--------+--------------------+
only showing top 20 rows



In [10]:
mapping = spark.read.load("./mapping.csv", format="csv",sep=",", inferschema="true", header="true")
genre_map = mapping.select("_c0", "0").rdd.collectAsMap()
genre_map

{0: 'Drama',
 1: 'Comedy',
 2: 'Romance Film',
 3: 'Thriller',
 4: 'Action',
 5: 'World cinema',
 6: 'Crime Fiction',
 7: 'Horror',
 8: 'Black-and-white',
 9: 'Indie',
 10: 'Action/Adventure',
 11: 'Adventure',
 12: 'Family Film',
 13: 'Short Film',
 14: 'Romantic drama',
 15: 'Animation',
 16: 'Musical',
 17: 'Science Fiction',
 18: 'Mystery',
 19: 'Romantic comedy'}

In [11]:
import pyspark.sql.functions as F

from pyspark.sql.types import *
def generateLabels(row, i): 
    label = 0
    for x in row[1:-1].split(","):
        if x.strip()[1:-1] == genre_map.get(i):
            label = 1
    return label
udfFunc = F.udf(generateLabels, IntegerType())
for i in range(len(genre_map)):    
    traindataframe = traindataframe.withColumn("label"+str(i), udfFunc("genre",F.lit(i)))
traindataframe.printSchema()

root
 |-- movie_id: long (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filteredwords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- label0: integer (nullable = true)
 |-- label1: integer (nullable = true)
 |-- label2: integer (nullable = true)
 |-- label3: integer (nullable = true)
 |-- label4: integer (nullable = true)
 |-- label5: integer (nullable = true)
 |-- label6: integer (nullable = true)
 |-- label7: integer (nullable = true)
 |-- label8: integer (nullable = true)
 |-- label9: integer (nullable = true)
 |-- label10: integer (nullable = true)
 |-- label11: integer (nullable = true)
 |-- label12: integer (nullable = true)
 |-- label13: integer (nullable = true)
 |-- label14: integer (nullable = true)
 |-- label15: integer (nullable = tr

In [12]:
def generateLabelCol(): 
    return ""
udfFunc = F.udf(generateLabelCol, StringType())

testdataframe = testdataframe.withColumn("predictions", udfFunc())
testdataframe.printSchema()

root
 |-- movie_id: long (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filteredwords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- predictions: string (nullable = true)



In [13]:
# from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
# from pyspark.mllib.regression import LabeledPoint
# from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

# def parsePoint(line):
#     return LabeledPoint(line.label, MLLibVectors.fromML(line.features))
# for i in range(len(genre_map)):
#     print('classifying Genre '+str(i+1)+ " : "+genre_map.get(i))
#     parsedData = traindataframe.selectExpr("label"+str(i)+" as label", "features").rdd.map(parsePoint)
#     model = LogisticRegressionWithLBFGS.train(parsedData)
#     model.save(spark,'./Models/Part3/model'+str(i))
#     labelsAndPreds = testdataframe.rdd.map(lambda p: (p.movie_id, model.predict(MLLibVectors.fromML(p.features))))
#     label_map = labelsAndPreds.collectAsMap()
#     def addLabel(m_id, row): 
#         row= row + str(label_map.get(m_id))+" "
#         return row
#     udfFunc = F.udf(addLabel, StringType())
#     testdataframe = testdataframe.withColumn("predictions", udfFunc("movie_id","predictions"))
    

In [14]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

def parsePoint(line):
    return LabeledPoint(line.label, MLLibVectors.fromML(line.features))
for i in range(len(genre_map)):
    print('classifying Genre '+str(i+1)+ " : "+genre_map.get(i))
    model = LogisticRegressionModel.load(spark,'./Models/Part3/model'+str(i))
    labelsAndPreds = testdataframe.rdd.map(lambda p: (p.movie_id, model.predict(MLLibVectors.fromML(p.features))))
    label_map = labelsAndPreds.collectAsMap()
    def addLabel(m_id, row): 
        row= row + str(label_map.get(m_id))+" "
        return row
    udfFunc = F.udf(addLabel, StringType())
    testdataframe = testdataframe.withColumn("predictions", udfFunc("movie_id","predictions"))
    

classifying Genre 1 : Drama
classifying Genre 2 : Comedy
classifying Genre 3 : Romance Film
classifying Genre 4 : Thriller
classifying Genre 5 : Action
classifying Genre 6 : World cinema
classifying Genre 7 : Crime Fiction
classifying Genre 8 : Horror
classifying Genre 9 : Black-and-white
classifying Genre 10 : Indie
classifying Genre 11 : Action/Adventure
classifying Genre 12 : Adventure
classifying Genre 13 : Family Film
classifying Genre 14 : Short Film
classifying Genre 15 : Romantic drama
classifying Genre 16 : Animation
classifying Genre 17 : Musical
classifying Genre 18 : Science Fiction
classifying Genre 19 : Mystery
classifying Genre 20 : Romantic comedy


In [15]:
testdataframe.select('movie_id','predictions').show(20, False)

+--------+----------------------------------------+
|movie_id|predictions                             |
+--------+----------------------------------------+
|1335380 |1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 |
|29062594|1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 |
|9252321 |1 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 |
|13455076|1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 |
|24165951|0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 |
|1925869 |1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 |
|10799612|1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 |
|28238240|0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 |
|17124781|1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 |
|28207941|1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 |
|19174305|1 0 1 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0 |
|18392317|1 0 1 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 1 0 |
|34420857|1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 1 1 |
|4039635 |1 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 |
|8034072 |1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 0 1 1 1 0 |
|4016437 |1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 |
|1520023 |1 

In [16]:
testdataframe.select('movie_id','predictions').write.format("csv").option("header", "true").mode("append").save("outputs/output3")